In [2]:
from typing import List
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from drn_interactions.transforms import SpikesHandler
from drn_interactions.spiketrains.spiketrain_stats import SpikeTrainDescriptor
from IPython.display import display
from drn_interactions.config import Config

sns.set_theme(context="talk", style="ticks")

%load_ext autoreload
%autoreload 2

In [3]:
sh = SpikesHandler(block="pre", bin_width=1, t_start=0, t_stop=1800)

df_spikes = sh.spikes

In [4]:
descriptor = SpikeTrainDescriptor()

df_stats = descriptor.describe(
    df_spikes, 
    t_start=0, 
    t_stop=1800, 
    burst_thresh=0.02, 
    max_fr_binsize=30, 
    cv_fr_binsize=30, 
    mfr_bin_size=30, 
    mfr_exclude_below=0.5,
    )

display(df_stats.sample(5))


c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\lib\nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,neuron_id,cv2_isi,is_buster,cv2_isi_burst,median_burst_interval,mean_firing_rate,mean_firing_rate_ifr,fraction_bursts,max_firing_rate,cv_firing_rate
311,1858,0.247980,False,0.218867,0.010550,7.393333,8.436490,0.010219,276,0.095914
456,2440,0.555387,False,0.454134,0.009850,8.785000,8.254651,0.052994,487,0.100593
332,1887,0.930190,False,0.892302,0.010000,0.846111,0.952403,0.020355,126,0.564093
79,114,1.264004,True,1.023163,0.003250,0.040556,1.037252,0.630137,63,0.208224
104,147,0.679095,False,0.603399,0.010633,7.356111,6.848458,0.053848,412,0.198251


In [25]:
df_stats.to_csv(Config.derived_data_dir / "spiketrain_stats.csv", index=False)

In [55]:
from binit import which_bin
from drn_interactions.spiketrains.spiketrain_stats import SpikeTrainStats




c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [91]:
from drn_interactions.spiketrains.spiketrain_stats import cv2

bins = np.arange(0, 1801, 30)
df_spikes["segment"] = which_bin(df_spikes["spiketimes"].values, bins, time_after=30)
stats = SpikeTrainStats(thresh_burst=0.02)
df_res = df_spikes.groupby(["neuron_id", "segment"])["spiketimes"].apply(stats)
df_res2 = df_res.reset_index().rename(columns={"level_2": "metric"}).assign(spiketimes=lambda x: x["spiketimes"].astype(float))

df_medians =df_res2.groupby(["neuron_id", "metric"])["spiketimes"].apply(lambda x: np.nanmedian(x)).reset_index()
df_medians = df_medians.pivot(index="neuron_id", columns="metric", values="spiketimes")
df_medians.reset_index().to_csv(Config.derived_data_dir / "spiketrain_stats_segments.csv", index=False)

c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\lib\nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [94]:

df_medians =df_res2.groupby(["neuron_id", "metric"])["spiketimes"].apply(lambda x: np.quantile(x, 0.25)).reset_index()
df_medians = df_medians.pivot(index="neuron_id", columns="metric", values="spiketimes")
df_medians.reset_index().to_csv(Config.derived_data_dir / "spiketrain_stats_segments_lower.csv", index=False)

In [92]:
from scipy.stats import variation
df_cvs =df_res2.fillna(method="backfill").fillna(method="bfill").groupby(["neuron_id", "metric"])["spiketimes"].apply(lambda x: variation(x)).reset_index()
df_cvs = df_cvs.pivot(index="neuron_id", columns="metric", values="spiketimes")
df_cvs.reset_index().to_csv(Config.derived_data_dir / "spiketrain_stats_volitility.csv", index=False)

c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\scipy\stats\stats.py:1144: RuntimeWarning: invalid value encountered in double_scalars
  return a.std(axis) / a.mean(axis)


In [93]:
df_cvs

metric,cv_isi,cv_isi_burst,frac_bursts,is_burst,mean_firing_rate,median_burst_interval
neuron_id,,,,,,
1,0.708062,0.703914,4.216790,3.937004,5.526981,2.196015
4,0.135616,0.189086,1.306272,1.595448,1.132500,0.554123
5,0.676700,0.717666,3.677958,5.744563,5.545973,0.971441
6,0.495110,0.508801,2.377892,NaN,0.629997,2.427074
8,0.298766,0.287441,2.434225,NaN,0.493682,1.842234
...,...,...,...,...,...,...
2627,0.164057,0.165351,3.348466,NaN,0.451154,3.440539
2628,0.398202,0.421166,1.226966,1.296849,0.551839,1.152353
2629,0.347679,0.346554,2.073357,5.385165,0.827169,1.299652
